In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [2]:
num_word=50000

In [3]:
path='https://github.com/duybluemind1988/NLP-with-Python/blob/master/data/ner_dataset.csv?raw=true'
df=pd.read_csv(path,encoding = "ISO-8859-1")
df = df[:num_word]
print(df.shape)
df.head()

(50000, 4)


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [4]:
df.isnull().sum()

Sentence #    47730
Word              0
POS               0
Tag               0
dtype: int64

In [5]:
df = df.fillna(method='ffill')
df

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
49995,Sentence: 2270,pushed,VBN,O
49996,Sentence: 2270,by,IN,O
49997,Sentence: 2270,South,NNP,B-geo
49998,Sentence: 2270,Africa,NNP,I-geo


Essential info about entities:
geo = Geographical Entity
org = Organization
per = Person
gpe = Geopolitical Entity
tim = Time indicator
art = Artifact
eve = Event
nat = Natural Phenomenon

We have 4,544 sentences that contain 10,922 unique words and tagged by 17 tags.

In [6]:
df['Sentence #'].nunique(), df.Word.nunique(), df.POS.nunique(), df.Tag.nunique()

(2270, 7464, 40, 17)

In [7]:
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [8]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-art,48
1,B-eve,39
2,B-geo,1490
3,B-gpe,968
4,B-nat,18
5,B-org,959
6,B-per,789
7,B-tim,880
8,I-art,27
9,I-eve,33


In [9]:
X = df.drop('Tag', axis=1)
X.head()

,Sentence #,Word,POS
0,Sentence: 1,Thousands,NNS
1,Sentence: 1,of,IN
2,Sentence: 1,demonstrators,NNS
3,Sentence: 1,have,VBP
4,Sentence: 1,marched,VBN


In [10]:
X.columns

Index(['Sentence #', 'Word', 'POS'], dtype='object')

In [11]:
X

,Sentence #,Word,POS
0,Sentence: 1,Thousands,NNS
1,Sentence: 1,of,IN
2,Sentence: 1,demonstrators,NNS
3,Sentence: 1,have,VBP
4,Sentence: 1,marched,VBN
...,...,...,...
49995,Sentence: 2270,pushed,VBN
49996,Sentence: 2270,by,IN
49997,Sentence: 2270,South,NNP
49998,Sentence: 2270,Africa,NNP


In [12]:
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
X.shape

(50000, 9774)

In [13]:
y = df.Tag.values
print(y.shape)
y

(50000,)


array(['O', 'O', 'O', ..., 'B-geo', 'I-geo', 'O'], dtype=object)

In [14]:
classes = np.unique(y)

In [15]:
classes = classes.tolist()
classes

['B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim',
 'O']

In [16]:
X.shape, y.shape

((50000, 9774), (50000,))

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)

In [18]:
X_train.shape, y_train.shape

((33500, 9774), (33500,))

### Perceptron

Because tag “O” (outside) is the most common tag and it will make our results look much better than they actual are. So we remove tag “O” when we evaluate classification metrics.

In [19]:
new_classes = classes.copy()
new_classes.pop()
new_classes

['B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim']

Out-of-core Algorithms

We will try some of the out-of-core algorithms that are designed to process data that is too large to fit into a single computer memory that support partial_fit method.

In [20]:
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(X_train, y_train, classes)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


-- Epoch 1
-- Epoch 1
Norm: 10.20, NNZs: 77, Bias: -4.000000, T: 33500, Avg. loss: 0.001433
Total training time: 0.73 seconds.
-- Epoch 1
Norm: 10.34, NNZs: 101, Bias: -3.000000, T: 33500, Avg. loss: 0.002030
Total training time: 0.76 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.8s


Norm: 50.47, NNZs: 1434, Bias: -5.000000, T: 33500, Avg. loss: 0.042269
Total training time: 0.78 seconds.
-- Epoch 1
Norm: 40.36, NNZs: 932, Bias: -3.000000, T: 33500, Avg. loss: 0.020746
Total training time: 0.76 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.6s


...

Norm: 24.43, NNZs: 417, Bias: -3.000000, T: 33500, Avg. loss: 0.010478
Total training time: 0.79 seconds.
-- Epoch 1
Norm: 8.66, NNZs: 63, Bias: -3.000000, T: 33500, Avg. loss: 0.001313
Total training time: 0.79 seconds.
-- Epoch 1
Norm: 6.16, NNZs: 27, Bias: -2.000000, T: 33500, Avg. loss: 0.000209
Total training time: 0.73 seconds.
-- Epoch 1
Norm: 38.47, NNZs: 860, Bias: -6.000000, T: 33500, Avg. loss: 0.023343
Total training time: 0.76 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    5.4s


Norm: 44.52, NNZs: 1248, Bias: -6.000000, T: 33500, Avg. loss: 0.028746
Total training time: 0.77 seconds.
-- Epoch 1
Norm: 21.84, NNZs: 341, Bias: -3.000000, T: 33500, Avg. loss: 0.009881
Total training time: 0.78 seconds.
Norm: 51.33, NNZs: 1478, Bias: 3.000000, T: 33500, Avg. loss: 0.046090
Total training time: 0.67 seconds.


[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:    6.8s finished


Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=5, n_iter_no_change=5, n_jobs=-1,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=10, warm_start=False)

In [21]:
#DNN with 50,000 words training
print(classification_report(y_pred=per.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

       B-art       0.75      0.17      0.27        18
       B-eve       0.00      0.00      0.00        14
       B-geo       0.76      0.39      0.52       476
       B-gpe       0.26      0.82      0.39       320
       B-nat       0.00      0.00      0.00         3
       B-org       0.56      0.45      0.50       301
       B-per       0.68      0.47      0.55       258
       B-tim       0.86      0.75      0.81       272
       I-art       0.00      0.00      0.00         5
       I-eve       0.67      0.36      0.47        11
       I-geo       0.36      0.60      0.45       115
       I-gpe       0.00      0.00      0.00        11
       I-nat       0.50      0.33      0.40         3
       I-org       0.81      0.23      0.36       200
       I-per       0.86      0.08      0.15       306
       I-tim       0.44      0.09      0.15        90

   micro avg       0.48      0.44      0.46      2403
   macro avg       0.47   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#From ebook with 100,000 words
print(classification_report(y_pred=per.predict(X_test), y_true=y_test, labels=new_classes))

             precision    recall  f1-score   support

      B-art       0.15      0.12      0.14        24
      B-eve       0.46      0.32      0.37        19
      B-geo       0.42      0.91      0.57      1085
      B-gpe       0.89      0.78      0.83       556
      B-nat       0.11      0.25      0.15        12
      B-org       0.55      0.35      0.43       589
      B-per       0.72      0.43      0.53       564
      B-tim       0.65      0.78      0.71       611
      I-art       0.02      0.08      0.03        12
      I-eve       0.00      0.00      0.00        18
      I-geo       0.81      0.32      0.46       230
      I-gpe       0.00      0.00      0.00        14
      I-nat       0.50      0.50      0.50         2
      I-org       0.71      0.41      0.52       445
      I-per       0.76      0.20      0.32       591
      I-tim       0.26      0.05      0.09       194

avg / total       0.62      0.55      0.53      4966



C:\Users\SusanLi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Linear classifiers with SGD training

In [22]:
sgd = SGDClassifier()
sgd.partial_fit(X_train, y_train, classes)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [23]:
print(classification_report(y_pred=sgd.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

       B-art       0.33      0.33      0.33        18
       B-eve       1.00      0.14      0.25        14
       B-geo       0.48      0.87      0.62       476
       B-gpe       0.86      0.56      0.68       320
       B-nat       0.00      0.00      0.00         3
       B-org       0.67      0.38      0.48       301
       B-per       0.70      0.48      0.57       258
       B-tim       0.72      0.72      0.72       272
       I-art       0.00      0.00      0.00         5
       I-eve       0.03      0.45      0.05        11
       I-geo       0.63      0.45      0.53       115
       I-gpe       0.00      0.00      0.00        11
       I-nat       0.00      0.00      0.00         3
       I-org       0.80      0.18      0.29       200
       I-per       0.79      0.37      0.51       306
       I-tim       0.55      0.07      0.12        90

   micro avg       0.57      0.52      0.54      2403
   macro avg       0.47   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Naive Bayes classifier for multinomial models

In [24]:
nb = MultinomialNB(alpha=0.01)
nb.partial_fit(X_train, y_train, classes)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [25]:
print(classification_report(y_pred=nb.predict(X_test), y_true=y_test, labels = new_classes))

              precision    recall  f1-score   support

       B-art       0.09      0.28      0.13        18
       B-eve       0.29      0.29      0.29        14
       B-geo       0.66      0.58      0.62       476
       B-gpe       0.65      0.73      0.68       320
       B-nat       0.18      0.67      0.29         3
       B-org       0.48      0.48      0.48       301
       B-per       0.38      0.48      0.42       258
       B-tim       0.60      0.72      0.65       272
       I-art       0.08      0.20      0.12         5
       I-eve       0.47      0.64      0.54        11
       I-geo       0.43      0.49      0.46       115
       I-gpe       0.00      0.00      0.00        11
       I-nat       0.00      0.00      0.00         3
       I-org       0.47      0.48      0.48       200
       I-per       0.52      0.45      0.49       306
       I-tim       0.16      0.26      0.20        90

   micro avg       0.50      0.54      0.52      2403
   macro avg       0.34   

### Passive Aggressive Classifier

In [26]:
pa =PassiveAggressiveClassifier()
pa.partial_fit(X_train, y_train, classes)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=1000, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [27]:
print(classification_report(y_pred=pa.predict(X_test), y_true=y_test, labels=new_classes))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00        18
       B-eve       0.00      0.00      0.00        14
       B-geo       0.87      0.14      0.25       476
       B-gpe       0.97      0.57      0.72       320
       B-nat       0.00      0.00      0.00         3
       B-org       0.18      0.92      0.30       301
       B-per       0.91      0.34      0.50       258
       B-tim       0.81      0.73      0.77       272
       I-art       0.00      0.00      0.00         5
       I-eve       0.00      0.00      0.00        11
       I-geo       0.80      0.21      0.33       115
       I-gpe       0.00      0.00      0.00        11
       I-nat       1.00      0.33      0.50         3
       I-org       0.69      0.06      0.10       200
       I-per       0.84      0.13      0.23       306
       I-tim       1.00      0.04      0.09        90

   micro avg       0.39      0.37      0.38      2403
   macro avg       0.50   

None of the above classifiers produced satisfying results. It is obvious that it is not going to be easy to classify named entities using regular classifiers.

### Conditional Random Fields (CRFs)

CRFs is often used for labeling or parsing of sequential data, such as natural language processing and CRFs find applications in POS Tagging, named entity recognition, among others.
sklearn-crfsuite
We will train a CRF model for named entity recognition using sklearn-crfsuite on our data set.

In [29]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 4.0MB/s 


In [30]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

#### Get sentences

In [31]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

In [32]:
df

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
49995,Sentence: 2270,pushed,VBN,O
49996,Sentence: 2270,by,IN,O
49997,Sentence: 2270,South,NNP,B-geo
49998,Sentence: 2270,Africa,NNP,I-geo


In [33]:
getter = SentenceGetter(df)
getter

In [40]:
sent = getter.get_next()
sent[:20]

[('The', 'DT', 'O'),
 ('protest', 'NN', 'O'),
 ('comes', 'VBZ', 'O'),
 ('on', 'IN', 'O'),
 ('the', 'DT', 'O'),
 ('eve', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('the', 'DT', 'O'),
 ('annual', 'JJ', 'O'),
 ('conference', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('Britain', 'NNP', 'B-geo'),
 ("'s", 'POS', 'O'),
 ('ruling', 'VBG', 'O'),
 ('Labor', 'NNP', 'B-org'),
 ('Party', 'NNP', 'I-org'),
 ('in', 'IN', 'O'),
 ('the', 'DT', 'O'),
 ('southern', 'JJ', 'O'),
 ('English', 'JJ', 'B-gpe')]

In [36]:
sentences = getter.sentences
sentences[:3]

[[('Thousands', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('demonstrators', 'NNS', 'O'),
  ('have', 'VBP', 'O'),
  ('marched', 'VBN', 'O'),
  ('through', 'IN', 'O'),
  ('London', 'NNP', 'B-geo'),
  ('to', 'TO', 'O'),
  ('protest', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('war', 'NN', 'O'),
  ('in', 'IN', 'O'),
  ('Iraq', 'NNP', 'B-geo'),
  ('and', 'CC', 'O'),
  ('demand', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('withdrawal', 'NN', 'O'),
  ('of', 'IN', 'O'),
  ('British', 'JJ', 'B-gpe'),
  ('troops', 'NNS', 'O'),
  ('from', 'IN', 'O'),
  ('that', 'DT', 'O'),
  ('country', 'NN', 'O'),
  ('.', '.', 'O')],
 [('Iranian', 'JJ', 'B-gpe'),
  ('officials', 'NNS', 'O'),
  ('say', 'VBP', 'O'),
  ('they', 'PRP', 'O'),
  ('expect', 'VBP', 'O'),
  ('to', 'TO', 'O'),
  ('get', 'VB', 'O'),
  ('access', 'NN', 'O'),
  ('to', 'TO', 'O'),
  ('sealed', 'JJ', 'O'),
  ('sensitive', 'JJ', 'O'),
  ('parts', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('the', 'DT', 'O'),
  ('plant', 'NN', 'O'),
  ('Wednesday', 'NNP', 'B-tim'),
  ('

#### Features extraction

Next, we extract more features (word parts, simplified POS tags, lower/title/upper flags, features of nearby words) and convert them to sklear-crfsuite format - each sentence should be converted to a list of dicts.

In [41]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

The above code were taken from sklearn-crfsuite official site.

Split train and test sets.

In [42]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [46]:
sentences[:1]

[[('Thousands', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('demonstrators', 'NNS', 'O'),
  ('have', 'VBP', 'O'),
  ('marched', 'VBN', 'O'),
  ('through', 'IN', 'O'),
  ('London', 'NNP', 'B-geo'),
  ('to', 'TO', 'O'),
  ('protest', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('war', 'NN', 'O'),
  ('in', 'IN', 'O'),
  ('Iraq', 'NNP', 'B-geo'),
  ('and', 'CC', 'O'),
  ('demand', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('withdrawal', 'NN', 'O'),
  ('of', 'IN', 'O'),
  ('British', 'JJ', 'B-gpe'),
  ('troops', 'NNS', 'O'),
  ('from', 'IN', 'O'),
  ('that', 'DT', 'O'),
  ('country', 'NN', 'O'),
  ('.', '.', 'O')]]

In [47]:
X[:1]

[[{'+1:postag': 'IN',
   '+1:postag[:2]': 'IN',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:word.lower()': 'of',
   'BOS': True,
   'bias': 1.0,
   'postag': 'NNS',
   'postag[:2]': 'NN',
   'word.isdigit()': False,
   'word.istitle()': True,
   'word.isupper()': False,
   'word.lower()': 'thousands',
   'word[-2:]': 'ds',
   'word[-3:]': 'nds'},
  {'+1:postag': 'NNS',
   '+1:postag[:2]': 'NN',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:word.lower()': 'demonstrators',
   '-1:postag': 'NNS',
   '-1:postag[:2]': 'NN',
   '-1:word.istitle()': True,
   '-1:word.isupper()': False,
   '-1:word.lower()': 'thousands',
   'bias': 1.0,
   'postag': 'IN',
   'postag[:2]': 'IN',
   'word.isdigit()': False,
   'word.istitle()': False,
   'word.isupper()': False,
   'word.lower()': 'of',
   'word[-2:]': 'of',
   'word[-3:]': 'of'},
  {'+1:postag': 'VBP',
   '+1:postag[:2]': 'VB',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:w

In [48]:
y[:2]

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-tim',
  'O',
  'O',
  'O',
  'B-org',
  'O',
  'O',
  'O',
  'O',
  'O']]

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [61]:
print(len(X_train)) #1520 sentences
print(len(X_train[0])) # first sentences have 31 words
print(len(X_train[0][0]))# first word have 15 features
print(len(X_train[0][1]))# second word have 19 features 
print(len(y_train)) # 1520 labels
print(len(y_train[0])) # 31 label for 31 words in first sentence
print(len(X_test))
print(len(X_test[0]))

1520
31
15
19
1520
31
750
21


In [58]:
X_train[:1]

[[{'+1:postag': 'VBD',
   '+1:postag[:2]': 'VB',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:word.lower()': 'cited',
   'BOS': True,
   'bias': 1.0,
   'postag': 'NNS',
   'postag[:2]': 'NN',
   'word.isdigit()': False,
   'word.istitle()': True,
   'word.isupper()': False,
   'word.lower()': 'officials',
   'word[-2:]': 'ls',
   'word[-3:]': 'als'},
  {'+1:postag': 'NN',
   '+1:postag[:2]': 'NN',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:word.lower()': 'warming',
   '-1:postag': 'NNS',
   '-1:postag[:2]': 'NN',
   '-1:word.istitle()': True,
   '-1:word.isupper()': False,
   '-1:word.lower()': 'officials',
   'bias': 1.0,
   'postag': 'VBD',
   'postag[:2]': 'VB',
   'word.isdigit()': False,
   'word.istitle()': False,
   'word.isupper()': False,
   'word.lower()': 'cited',
   'word[-2:]': 'ed',
   'word[-3:]': 'ted'},
  {'+1:postag': 'NNS',
   '+1:postag[:2]': 'NN',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1

In [62]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [63]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=new_classes)

0.7467577210509984

In [64]:
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

              precision    recall  f1-score   support

       B-art       0.50      0.05      0.10        19
       B-eve       0.71      0.29      0.42        17
       B-geo       0.67      0.86      0.75       475
       B-gpe       0.83      0.73      0.78       334
       B-nat       0.00      0.00      0.00         5
       B-org       0.71      0.57      0.63       354
       B-per       0.79      0.79      0.79       243
       B-tim       0.87      0.86      0.87       291
       I-art       0.00      0.00      0.00        12
       I-eve       1.00      0.25      0.40        16
       I-geo       0.65      0.65      0.65        92
       I-gpe       0.00      0.00      0.00         9
       I-nat       0.00      0.00      0.00         1
       I-org       0.77      0.76      0.76       254
       I-per       0.84      0.93      0.88       283
       I-tim       0.71      0.62      0.66        88

   micro avg       0.76      0.75      0.76      2493
   macro avg       0.56   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.grid_search import RandomizedSearchCV

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=new_classes)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

C:\Users\SusanLi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\SusanLi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 17.0min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=None, c2=None,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error...e,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False),
          fit_params={}, iid=True, n_iter=50, n_jobs=-1,
          param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000022C766AD048>, 'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000022C76751978>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=make_scorer(flat_f1_score, average=weighted, labels=['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim']),
   

In [ ]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.0036898984638244928, 'c2': 0.11585183551331574}
best CV score: 0.7737211773297741
model size: 1.30M


In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels=new_classes))

C:\Users\SusanLi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

      B-art       1.00      0.03      0.07        29
      B-eve       0.83      0.21      0.33        24
      B-geo       0.75      0.87      0.81      1043
      B-gpe       0.88      0.78      0.83       588
      B-nat       0.67      0.20      0.31        10
      B-org       0.74      0.63      0.68       649
      B-per       0.81      0.80      0.81       546
      B-tim       0.90      0.84      0.87       589
      I-art       0.00      0.00      0.00         7
      I-eve       0.67      0.22      0.33        18
      I-geo       0.67      0.71      0.69       204
      I-gpe       0.39      0.53      0.45        17
      I-nat       1.00      0.50      0.67         2
      I-org       0.78      0.72      0.75       545
      I-per       0.81      0.89      0.85       574
      I-tim       0.79      0.66      0.72       185

avg / total       0.80      0.78      0.78      5030



In [65]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-geo  -> I-geo   6.025124
B-per  -> I-per   5.752572
I-tim  -> I-tim   5.537304
B-org  -> I-org   5.377582
I-org  -> I-org   5.225950
B-tim  -> I-tim   5.203619
B-gpe  -> I-gpe   4.811577
I-per  -> I-per   4.570931
B-art  -> I-art   4.515261
B-eve  -> I-eve   4.346047
I-gpe  -> I-gpe   4.146536
I-geo  -> I-geo   4.096209
I-art  -> I-art   3.888031
O      -> O       3.750474
B-nat  -> I-nat   3.301682
I-eve  -> I-eve   3.097023
B-org  -> B-art   2.426228
I-nat  -> I-nat   1.998332
O      -> B-eve   1.883422
O      -> B-per   1.803690

Top unlikely transitions:
I-geo  -> B-per   -1.034029
B-org  -> I-per   -1.076155
I-tim  -> B-tim   -1.083762
B-geo  -> I-org   -1.174412
B-geo  -> I-per   -1.183731
B-gpe  -> I-org   -1.188898
I-org  -> I-per   -1.284712
B-org  -> B-org   -1.326719
B-gpe  -> I-geo   -1.380641
B-tim  -> B-tim   -1.421861
O      -> I-art   -1.550078
B-tim  -> B-gpe   -1.830413
B-geo  -> B-per   -2.019120
B-per  -> B-per   -2.047799
B-gpe  -> B-gpe  

It is very likely that the beginning of a geographical entity (B-geo) will be followed by a token inside geographical entity (I-geo), but transitions to inside of an organization name (I-org) from tokens with other labels are penalized hugely.

In [66]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
5.507773 B-tim    word[-2:]:0s
4.825628 B-tim    word[-3:]:day
4.656911 O        bias
4.545841 O        BOS
3.855805 B-tim    word[-2:]:ay
3.588437 B-gpe    word.istitle()
3.468236 O        word.lower():jewish
3.242285 O        word.lower():kurdish
3.134375 B-tim    +1:word.lower():last
3.091690 I-tim    word[-3:]:day
2.953323 B-org    +1:word.lower():interim
2.876845 B-org    word.lower():al-qaida
2.871956 B-tim    word[-3:]:ber
2.845636 B-org    word.lower():parliament
2.814774 B-org    -1:word.lower():extremist
2.795836 B-gpe    word.lower():bosnia-herzegovina
2.775126 B-geo    -1:word.lower():sponsored
2.765393 B-per    word.lower():prime
2.712992 B-geo    +1:word.lower():province
2.685214 B-geo    +1:word.lower():palestinian
2.680229 O        -1:word.lower():prime
2.657546 B-per    word.lower():obama
2.655386 O        word[-2:]:ty
2.655353 B-gpe    -1:word.lower():recognize
2.638968 B-tim    word.lower():midnight
2.629440 B-org    -1:word.lower():nepal
2.616242 B-org

Observations: 

1). __```5.183603 B-tim word[-3]:day```__
The model learns that if a nearby word was “day” then the token is likely a part of a Time indicator.

2). __```3.370614 B-per word.lower():president```__
The model learns that token "president" is likely to be at the beginning of a person name.

3). __```-3.521244 O postag:NNP```__
The model learns that proper nouns are often entities.

4). __```-3.087828 O word.isdigit()```__
Digits are likely entities.

5). __```-3.233526 O word.istitle()```__
TitleCased words are likely entities.

### ELI5

ELI5 is a Python package which helps to debug machine learning classifiers and explain their predictions. ELI5 allows to check weights of sklearn_crfsuite.CRF models.

In [ ]:
import eli5

eli5.show_weights(crf, top=10)

From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,3.465,0.477,-2.326,0.973,-2.034,0.919,-4.235,0.506,-1.857,0.049,-1.256,0.794,-4.544,2.058,-3.123,1.417,-4.153
B-art,-0.876,-0.023,4.951,-0.003,-0.101,-0.373,-0.232,-0.373,-0.251,-0.008,-0.08,0.606,-0.601,-0.816,-0.784,-0.669,-0.324
I-art,-0.986,-0.279,4.665,-0.014,-0.086,0.336,-0.262,-0.272,-0.089,-0.008,-0.066,-0.44,-0.52,-0.747,-0.563,0.093,-0.399
B-eve,-0.533,-0.006,-0.077,-0.022,4.847,-0.234,-0.219,-0.328,-0.177,0.0,-0.04,-0.479,-0.504,-0.844,-0.409,-0.656,-0.515
I-eve,-0.333,0.0,-0.034,-0.653,3.968,-0.257,-0.193,-0.105,-0.059,-0.01,-0.009,-0.233,-0.272,-0.351,-0.387,-0.384,-0.177
B-geo,0.216,1.413,-1.024,-0.136,-0.695,-1.541,6.008,1.1,-1.881,-0.05,-0.502,-1.03,-1.947,-0.966,-1.813,1.688,-1.373
I-geo,-0.034,-0.048,-0.417,-0.029,-0.256,-1.011,5.296,-0.468,-0.719,-0.009,-0.147,-0.786,-1.018,-0.791,-0.642,1.238,-0.928
B-gpe,0.62,-0.255,-0.858,-0.278,-0.661,-0.184,-2.152,-3.169,3.92,-0.049,-0.296,0.951,-1.848,0.572,-1.357,-0.347,-0.987
I-gpe,-0.656,-0.163,-0.082,-0.01,-0.031,-0.007,-0.61,-0.624,3.97,0.0,-0.024,-0.377,-0.622,-0.619,-0.441,-0.684,-0.247
B-nat,-0.405,-0.001,-0.055,0.0,-0.042,-0.254,-0.109,-0.182,-0.068,-0.005,3.208,-0.255,-0.334,-0.55,-0.394,-0.231,-0.078


It does make sense that I-entity must follow B-entity, such as I-geo follows B-geo, I-org follows B-org, I-per follows B-per, and so on. 

We can also see that it is not common in this dataset to have a person right after an organization name (B-org -> I-per has a large negative weight).

If we regularize CRF more, we can expect that only features which are generic will remain, and memoized tokens will go. Let’s check what effect does regularization have on CRF weights:

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=200,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train)
eli5.show_weights(crf, top=10)

From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,1.782,0.0,0.0,0.0,0.0,1.456,0.0,0.303,0.0,0.0,0.0,0.791,0.0,0.062,0.0,1.709,0.0
B-art,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-art,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-eve,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-eve,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-geo,0.23,0.0,0.0,0.0,0.0,0.0,2.704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-geo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-gpe,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-gpe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-nat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
)
crf.fit(X_train, y_train);
eli5.show_weights(crf, top=5, show=['transition_features'])

From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,3.561,0.747,-2.344,0.928,-1.847,1.1,-4.541,0.523,-1.684,0.239,-1.025,0.795,-4.779,1.64,-3.017,1.39,-4.202
B-art,-0.556,0.0,5.467,0.0,0.0,-0.021,-0.085,-0.145,0.0,0.0,0.0,0.478,-0.328,-0.735,-0.394,-0.41,0.0
I-art,-0.74,0.0,5.438,0.0,0.0,0.509,-0.075,-0.079,0.0,0.0,0.0,-0.091,-0.315,-0.777,-0.246,0.19,-0.062
B-eve,-0.213,0.0,0.0,0.0,5.415,0.0,0.0,-0.12,0.0,0.0,0.0,-0.123,-0.319,-0.662,-0.397,-0.512,-0.193
I-eve,-0.239,0.0,0.0,-0.279,4.838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.013,-0.215,-0.002,-0.236,0.0
B-geo,0.31,1.357,-0.737,0.0,-0.498,-1.395,5.227,0.733,-1.392,0.0,-0.233,-0.973,-1.993,-1.367,-1.873,1.648,-1.25
I-geo,-0.01,0.0,-0.123,0.0,0.0,-0.88,4.457,-0.486,-0.606,0.0,0.0,-0.766,-0.944,-1.214,-0.579,1.036,-0.96
B-gpe,0.623,0.0,-0.581,-0.029,-0.427,-0.116,-2.327,-3.359,4.986,0.0,0.0,0.901,-1.945,0.294,-1.231,-0.431,-0.876
I-gpe,-0.21,0.0,0.0,0.0,0.0,0.24,-0.291,-0.233,4.878,0.0,0.0,-0.101,-0.352,-0.385,-0.263,-0.379,0.0
B-nat,-0.402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.694,0.0,-0.016,-0.48,-0.194,0.0,0.0


The model learned large negative weights for impossible transitions like O -> I-geo, O -> I-org and O -> I-tim, and so on.

In order to easy to read, we can check only a subset of tags.

In [ ]:
eli5.show_weights(crf, top=10, targets=['O', 'B-org', 'I-per'])

Or check only some of the features for all tags.

In [ ]:
eli5.show_weights(crf, top=10, feature_re='^word\.is',
                  horizontal_layout=False, show=['targets'])

Weight?,Feature
-2.186,word.isupper()
-2.720,word.isdigit()
-3.387,word.istitle()
Weight?,Feature
+0.151,word.istitle()
-0.214,word.isupper()
Weight?,Feature
+0.607,word.istitle()
+0.597,word.isdigit()
Weight?,Feature
